## Make your own heatmap based on Strava activities
This notebook shows you how to create your own heatmap based on your Strava activities.

You need to create a Strava API application in order to use their API. Follow the instructions on this page to create your app: <https://medium.com/@annthurium/getting-started-with-the-strava-api-a-tutorial-f3909496cd2d>

After setting up the app, note down the following information (you will need it to run this notebook):
- Client id
- Client secret

**Note:** Strava imposes some request limits (30'000/day, and 600/every 15min). 

In [ ]:
!pip install stravaio folium

In [ ]:
import os
import logging
import json
import urllib
import requests
import folium
from stravaio import StravaIO

In [ ]:
# Paste your client id and client secret here.
STRAVA_CLIENT_ID = "ENTER-YOUR-CLIENT-ID"
STRAVA_CLIENT_SECRET = "ENTER-YOUR-CLIENT-SECRET"

### Authorization with Strava
The cell below creates the proper authorization link using the Stravaio Python library, which is used later to retrieve activities.
It is important to run this cell, just pasting the access_token from your Strava settings will not work, because Stravaio needs to be authorized.

- Run the cell below and click the link that is printed, when prompted click "Authorize" on the website that opens
- After you click "Authorize" you see something like, "This site can't be reached"
- Stay on that page and look at the URL
- The URL will show the authorization code (the bit after "code=" in the URL) and scope you accepted
- Copy the code and paste it below and continue the notebook execution

More detailed info can be found here:
- <https://developers.strava.com/docs/getting-started/>
- <https://developers.strava.com/docs/authentication/>

In [ ]:
params_oauth = {
    "client_id": STRAVA_CLIENT_ID,
    "response_type": "code",
    "redirect_uri": f"http://localhost:8000/authorization_successful",
    "scope": "read,profile:read_all,activity:read",
    "state": 'https://github.com/sladkovm/strava-http', # Sladkovm is the author of the Stravaio library
    "approval_prompt": "force"
}
values_url = urllib.parse.urlencode(params_oauth)
base_url = 'https://www.strava.com/oauth/authorize'
authorize_url = base_url + '?' + values_url
print(authorize_url)

In [ ]:
# Paste the code from the URL here. Afterwards there are no manual steps anymore.
AUTHORIZATION_CODE = "ENTER-YOUR-AUTHORIZATION-CODE"

The following cell retrieves an access token using the authorization code. That access token can then be used to retrieve Strava data.

In [ ]:
payload = {
    "client_id": STRAVA_CLIENT_ID,
    "client_secret": STRAVA_CLIENT_SECRET,
    "grant_type": "authorization_code",
    "code": AUTHORIZATION_CODE,
}

response = requests.request(
    "POST", "https://www.strava.com/api/v3/oauth/token", data=payload
)

response = json.loads(response.text)
TOKEN = response["access_token"]

In [ ]:
!pip install stravaio folium

In [ ]:
client = StravaIO(access_token=TOKEN)
athlete = client.get_logged_in_athlete()
activities = client.get_logged_in_athlete_activities(after=20170101)

In [ ]:
m = folium.Map(
    tiles="cartodbpositron",
    location=[59.925, 10.728123],
    zoom_start=11.5,
    control_scale=True
)
folium.TileLayer("cartodbpositron").add_to(m)
folium.TileLayer("cartodbdark_matter").add_to(m)
folium.LayerControl().add_to(m)

In [ ]:
def downsample(l, n):
    """Returns every nth element from list l. Returns the
    original list if n is set to 1.
    Used to reduce the number of GPS points per activity,
    to improve performance of the website.
    """
    
    return l[0::n]

def map_activities(activities, folium_map, opacity=0.5, weight=1):
    if len(activities) == 0:
        logging.info("No activities found, returning empty folium map.")
        return folium_map

    counter = 0
    for a in activities:
        if a.type == "Workout":
            continue
        streams = client.get_activity_streams(a.id, athlete.id)
        try:
            points = list(zip(streams.lat, streams.lng))
            points = downsample(l=points, n=2)
            if a.type == "Run":
                folium.PolyLine(
                    locations=points, color="#ff9933", opacity=opacity, weight=weight
                ).add_to(folium_map)
            elif a.type == "Ride":
                folium.PolyLine(
                    locations=points, color="#0066ff", opacity=opacity, weight=weight
                ).add_to(folium_map)
            elif a.type == "NordicSki":
                folium.PolyLine(
                    locations=points, color="#00ffff", opacity=opacity, weight=weight
                ).add_to(folium_map)
            elif a.type == "AlpineSki":
                folium.PolyLine(
                    locations=points, color="#00ccff", opacity=opacity, weight=weight
                ).add_to(folium_map)
            elif a.type == "Canoeing":
                folium.PolyLine(
                    locations=points, color="#00ff55", opacity=opacity, weight=weight
                ).add_to(folium_map)
            elif a.type == "IceSkate":
                folium.PolyLine(
                    locations=points, color="#f6ff00", opacity=opacity, weight=weight
                ).add_to(folium_map)
            else:
                folium.PolyLine(
                    locations=points, color="#cc00ff", opacity=opacity, weight=weight
                ).add_to(folium_map)
            logging.critical("Mapped activity with id: {}".format(a.id))
        except Exception:
            logging.error("Could not map activity with id: {}".format(a.id))
            
    return folium_map

In [ ]:
m = map_activities(
    activities=activities,
    folium_map=m,
    opacity=0.5,
    weight=2
)

In [ ]:
m